<span style="color:#888888">Copyright (c) 2014-2021 National Technology and Engineering Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525 with National Technology and Engineering Solutions of Sandia, LLC, the U.S. Government retains certain rights in this software.     Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="color:#888888">1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.</span>

<span style="color:#888888">2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.</span>

<span style="color:#888888">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# <span style="color:#0054a8">**Demo 3:**</span> <span style="color:#555555">Travel Prediction Using Alignment</span>

This notebook describes how to use well-aligned historical trajectories to predict qualities about an observed trajectory's travel. Namely, in this notebook we predict the origin/destination location of an observed trajectory and the location of an observed trajectory in a specified amount of time. 

In [ ]:
from tracktable.applications.prediction import *
from tracktable.core import data_directory
import os.path

## Load in the Data

We read in the file containing historical trajectories and store all relevant information in a dictionary. In our case, the dataset is a ~500 trajectory subset from one day of flight data over the US airspace. While using the full day of flight data (or even more data) would yield better predictions, we use a subset to cut down on the time required to process historical trajectories. 

In [ ]:
historical_data_file = os.path.join(data_directory(), 'prediction_historical_trajectories.traj')

In [ ]:
prediction_dictionary = process_historical_trajectories(historical_data_file,  # file containing historical trajectories                                                                                          
                                               separation_time=20, # minutes
                                               separation_distance=100, # km 
                                               minimum_length=20, # km 
                                               minimum_total_distance=200, # km
                                               only_commercial=True # relevant for flight data
                                               )

Read in the file containing the observed trajectories into a list

In [ ]:
observed_data_file = os.path.join(data_directory(), 'prediction_observed_trajectories.traj')

observed_trajectories = TrajectoryReader()
observed_trajectories.input = open(observed_data_file, 'r')
observed_trajectories = list(observed_trajectories)

## Visualize the Dataset

The number of trajectories in the historical data is...

In [ ]:
len(prediction_dictionary['trajectories'])

An image of the historical data is shown below. For brevity, we upload an image of the dataset. However, if you wish to render an interactive version of the dataset, uncomment and run the code below.

In [ ]:
# render_trajectories(prediction_dictionary['trajs'], backend='folium', tiles='CartoDBPositron')

![prediction_data](demo_images/prediction_data.png)

The number of observed trajectories is....

In [ ]:
len(observed_trajectories)

We visualize them below.

In [ ]:
render_trajectories(observed_trajectories, backend='folium', tiles='CartoDBPositron')

## Prediction

### Step 1: choose the observed trajectory

Choose and observed trajectory from the list.

In [ ]:
observed_trajectory = observed_trajectories[0]

Choose how many sample points to represent the observed trajectory with. 
<br><br>
The sample points (samples) are evenly spaced points along the observed trajectory. The first and last sample points in the list are the end points of the observed trajectory. The fewer the sample points, the less detailed the representation of the observed trajectory. For example, if 2 sample points are used, the observed trajectory is represented as a line segment. Change this number from 2 to 4 to see the difference in representation. 

In [ ]:
sampled_trajectory = sample_trajectory(observed_trajectory, 2)

In [ ]:
render_trajectories([observed_trajectory, sampled_trajectory], line_color=['blue', 'red'], backend='folium', tiles='CartoDBPositron', show_points=True)

### Step 2: Alignment

Alignment is the central, driving algorithm of prediction. It finds well-aligned historical trajectories that are similar in shape and relative location to an observed trajectory. These well-aligned historical trajectories can be used to fill in unknown qualities about an object's motion (observed trajectory). Two applications of alignment are origin/destination prediction and time-based location prediction. We show those here.

![prediction_cartoon](demo_images/prediction_cartoon.png)

A historical trajectory is well-aligned with an observed trajectory if for each sample point of the observed trajectory, the distance to the historical trajectory is less than the neighborhood distance (radius of circles in image above). The orange historical trajectory is well aligned with the black historical trajectory above. The align function returns a dictionary which maps each historical trajectory to the number of sample points it is close to. We then filter this dictionary to contain only the trajectories that are close to all of the sample points. 

In [ ]:
neighborhood_distance = 10

In [ ]:
well_aligned_trajectories = find_well_aligned_trajectories(sampled_trajectory, prediction_dictionary, neighborhood_distance)

### Step 3: Direction check

![prediction_cartoon_2](demo_images/prediction_cartoon_2.png)

To determine if a historical trajectory is going in the same direction as an observed trajectory, first, represent the historical trajectory using segments. Find the closest segment of the historical trajectory to the first and last points in time of the observed trajectory. Interpolate along these two segments to find the closest point in space on the historical trajectory to the first and last point of the observed trajectory. Let t1 be the time of the point on the historical trajectory closest to the first point of the observed trajectory in space. Let t2 be the time of the point on the historical trajectory closest to the last point of the observed trajectory in space. If t2 - t1 > 0 then the historical trajectory and the observed trajectory are going in the same direction.

In [ ]:
same_direction = find_same_direction_trajectories(sampled_trajectory, well_aligned_trajectories, prediction_dictionary)

### Step 4: Assign a weight to each trajectory  

The weight assigned to a trajectory is a function of the distance of the sample point that is farthest away from that trajectory. Here, a trajectory's weight is this distance scaled linearly by the neighborhood distance (1 is the weight if the trajectory and sample point overlap, 0 is the weight if the distance from the trajectory to the sample point is neighborhood distance). 

In [ ]:
weight_function = lambda d: 1 - d / neighborhood_distance 

In [ ]:
weights = find_weights_trajectories(sampled_trajectory, same_direction, prediction_dictionary, weight_function)

Optionally, trajectories can be grouped by origin/destination pair during this step. Weights are then assigned to each origin/destination pair. This is not shown above, but is used for origin/destination prediction.

## Make Predictions

Use the two prediction functions to link all of these steps together. 

### Origin/Destination prediction

We can make a prediction about the observed trajectory's orign and destination. On this list of predictions, a higher weight corresponds to a more likely prediction. Optionally, set the neighborhood distance (how close well-aligned historical trajectories must be to the sample points of the observed trajectory). 

In [ ]:
results = predict_origin_destination(observed_trajectory, 
                                     prediction_dictionary,
                                     samples=4,
                                     neighbor_distance=10 #km
                                    )

We can also visualize the trajectories leading to this prediction. The observed trajectory is displayed in red. The trajectories from the historical dataset that passed nearby the observed trajectory are displayed in colors corresponding to their position on the prediction list, as shown below:

In [ ]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

plt.figure(figsize=(20, 2))
ax = plt.axes()
ax.get_xticklabels()
ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap('viridis'))
ax.tick_params(labelbottom=False, labelleft=False, bottom=False,
                left=False)
plt.title('lower', fontsize=20, loc='left')
plt.xlabel('higher', fontsize=20, loc='right')

plt.show()

In [ ]:
# origin_destination_render(observed_trajectory, results, prediction_dictionary)

# pick one trajectory to render for each OD pair
possibilities = []
for prediction in results['predictions']:
    trajectories = results['OD_pairs_to_trajs'][prediction]
    possibilities.append(prediction_dictionary['trajectories']
                            [prediction_dictionary['id_to_index'][trajectories[0]]])

# render trajectories in different colors
cmap = matplotlib.cm.get_cmap('viridis')
colors = []
for x in range(0, len(possibilities)):
    rgb = cmap(x * (1 / len(possibilities)))[:3]
    colors.append(matplotlib.colors.to_hex(rgb))
# reverse so that more likely predictions have lighter colors
colors.reverse()

# make the sub trajectory red
possibilities.append(observed_trajectory)
colors.append('red')
map_canvas = render_trajectories(possibilities, line_color=colors, show=True, save=False,
                    backend='folium', tiles='CartoDBPositron', filename='pred_results.html',
                    attr='.', crs='EPSG3857')

### Time-based location prediction

Time-based location prediction is done in a similar way to the direction check. First, represent a well-aligned historical trajectory using segments. Find the closest segment of the historical trajectory to the last point in time of the observed trajectory. Interpolate along this segment to find the closest point in space on the historical trajectory to the last point of the observed trajectory. Find the point on the historical trajectory that is x minutes from this point. This is the predicted location in x minutes and the portion of the historical trajectory between these two points is its predicted path. 

![prediction_cartoon_3](demo_images/prediction_cartoon_3.png)

We can make a prediction about where the observed object will be in a given number of minutes (in this case 30). Optionally, set the neighborhood distance (how close well-aligned historical trajectories must be to the sample points of the observed trajectory).  

In [ ]:
points, paths, weights = predict_location(observed_trajectory, prediction_dictionary, minutes=30, samples=4, neighbor_distance=10)

Again, we can visualize the historical trajectories leading to this prediction. The trajecories from the historical dataset that passed nearby the observed trajectory are displayed in grey. They represent the possible future paths of the observed trajectory. The white dots represent the possible locations the observed trajectory will be in the specified amount of time. The color of the heatmap corresponds to the likelihood the observed object will be in a given region in the specified amount of time, as shown below:

In [ ]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

plt.figure(figsize=(20, 2))
ax = plt.axes()
ax.get_xticklabels()
ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap('viridis'))
ax.tick_params(labelbottom=False, labelleft=False, bottom=False,
                left=False)
plt.title('lower', fontsize=20, loc='left')
plt.xlabel('higher', fontsize=20, loc='right')

plt.show()

In [ ]:
def heat_map_helper(points, weights):
    """Creates the approriate point formatting to use a heatmap

    Arguments:
        points: list of points to format
        weights: the weights those points received in the prediction algorithm

    Returns: a list of properly formatted points to feed into a heatmap
    """

    return [[points[key][1], points[key][0], weights[key]]
            for key in points.keys()]

In [ ]:
# pos_heatmap(points, paths, weights, observed_trajectory)

# create the heat map
heat_map = folium.Map(tiles='CartoDBPositron', zoom_start=4)

# lat, long, weight of points to render
display_points = heat_map_helper(points, weights)
to_render = list(paths.values())

# group all trajectories (including observed trajectory) in one list for rendering
to_render.append(observed_trajectory)
colors = ['grey'] * len(paths)
colors.append('red')
gradient = matplotlib_cmap_to_dict('viridis')
heat_map = render_trajectories(to_render, map_canvas=heat_map, backend='folium',
                                line_color=colors, linewidth=1.0,
                                tiles='CartoDBPositron', attr='.', crs='EPSG3857')
HeatMap(display_points, gradient=gradient).add_to(heat_map)

heat_map

## Create a small relevant data file.

First, make a list of relevant trajectories. This can be done in two ways for predictions:

In [ ]:
relevant_trajectories = find_relevant_trajectories_origin_destination(results, prediction_dictionary)

In [ ]:
relevant_trajectories = find_relevant_trajectories_location(points, prediction_dictionary)

Then, write these trajectories to a .traj file.

In [ ]:
write_trajectories('subset.traj', relevant_trajectories)

Now, instead of loading from a large historical data file, you can load only the trajectories used in the prediction. 